# POC

## Imports

In [136]:
import itertools
import json
import os
import re
import tempfile
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import dotenv
import pandas as pd
import pytesseract
import requests
from openpyxl import load_workbook
from pdf2image import convert_from_path
from PIL import Image
from tqdm import tqdm

## Vars

In [3]:
PATH_DOCS = Path("../docs")
PATH_TEMPLATE = Path("../templates")
PATH_OUTPUTS = Path("../outputs")

## Claude test

In [4]:
dotenv.load_dotenv("../keys.sh", override=True)

python-dotenv could not parse statement starting at line 1


True

In [5]:
CLAUDE_KEY = os.environ["CLAUDE_KEY"]

In [6]:
class ClaudeClient:
    """A simple client for the Anthropic Claude API."""
    
    def __init__(self, api_key: str):
        """Initialize the Claude client with your API key."""
        self.api_key = api_key
        
        self.base_url = "https://api.anthropic.com/v1"
        self.headers = {
            "x-api-key": self.api_key,
            "anthropic-version": "2023-06-01",
            "content-type": "application/json"
        }
    
    def create_message(
        self,
        model: str = "claude-3-7-sonnet-20250219",
        messages: List[Dict[str, str]] = None,
        system: Optional[str] = None,
        max_tokens: int = 1024,
        temperature: float = 0.7,
        stream: bool = False
    ) -> Dict[str, Any]:
        """
        Create a message using the Claude API.
        
        Args:
            model: The Claude model to use
            messages: List of message objects with role and content
            system: Optional system prompt to guide Claude's behavior
            max_tokens: Maximum number of tokens to generate
            temperature: Controls randomness (0-1)
            stream: Whether to stream the response
            
        Returns:
            API response as a dictionary
        """
        if not messages:
            raise ValueError("Messages are required")
            
        url = f"{self.base_url}/messages"
        
        payload = {
            "model": model,
            "messages": messages,
            "max_tokens": max_tokens,
            "temperature": temperature,
            "stream": stream
        }
        
        if system:
            payload["system"] = system
            
        response = requests.post(url, headers=self.headers, json=payload)
        
        if response.status_code != 200:
            raise Exception(f"API request failed with status {response.status_code}: {response.text}")
            
        return response.json()


In [7]:
client = ClaudeClient(CLAUDE_KEY)

In [ ]:
system = """
    Renvoie tes réponses sous format json.
    Extrait toutes les informations relatives aux différents parties.
"""

cell_value = "TRIBUNAL JUDICIAIRE DE DIEPPE Service chargé du contrôle des expertises judiciaires Tél. : 02.76.72.97.34 Référence à rappeler dans tous courriers : Mesure d'instruction : 24/173 N° de : 24/00052 N° Portalis DB2U-W-B7I-DDMR Ordonnance de Référé du 05 Juin 2024 DEMANDEURS M. Claude MASSERE Mme Françoise, Brigitte, Jeanne, Julia LAMAILLE épouse MASSERE représentés par la SCP MORIVAL AMISSE MABIRE, Avocats associés au Barreau de DIEPPE DÉFENDEURS La S.A.R.L. BRUGOT XAVIER représentée par la SCP LENGLET, MALBESIN & Associés, Avocats associés au Barreau de ROUEN La S.A. AXA France IARD représentée par la SCP LENGLET, MALBESIN & Associés, Avocats associés au Barreau de ROUEN M. Olivier BOUDET représenté par la SELARL PATRICE LEMIEGRE PHILIPPE FOURDRIN SUNA GUNEY & Associés, Avocats associés au Barreau de ROUEN La Société MUTUELLE DES ARCHITECTES FRANCAIS non représentée Monsieur, DIEPPE, le 21 Août 2024. Monsieur Franck HIBON 6 Impasse des Hautes Terres Les Vertus franck.hibon.expert-de-justice.or2 76550 SAINT AUBIN SUR SCIE EXPERTISE JUDICIAIRE, AVEC CONSIGNATION Date limite de dépôt du rapport d'expertise judiciaire : 28 Février 2025 J'ai l'honneur de vous adresser sous ce pli copie d'une décision qui vous a commis. re IMPORTANT : article 267 du Code de Procédure Civile - Décret n° 89-511 du 20 juillet 1989, article 3, en vigueur le 15 septembre 1989 : Il vous appartient de faire connaître au service chargé du contrôle des expertises judiciaires votre acceptation ou votre refus dès réce fion de la résente. Dans le cas où vous ne pourriez accepter cette mission, veuillez renvoyer l'ensemble des documents au service chargé du contrôle des expertises judiciaires."

messages = [
    {"role": "user", "content": cell_value}
]

response = client.create_message(
    model="claude-3-7-sonnet-20250219",
    messages=messages,
    system=system,
    max_tokens=500,
    temperature=0.7
)

# Print the response
print("Claude's response:")
print(response["content"][0]["text"])

Claude's response:
```json
{
  "tribunal": "TRIBUNAL JUDICIAIRE DE DIEPPE",
  "service": "Service chargé du contrôle des expertises judiciaires",
  "contact": "Tél. : 02.76.72.97.34",
  "references": {
    "mesure_instruction": "24/173",
    "numero_dossier": "24/00052",
    "numero_portalis": "DB2U-W-B7I-DDMR"
  },
  "decision": "Ordonnance de Référé du 05 Juin 2024",
  "parties": {
    "demandeurs": [
      {
        "nom": "M. Claude MASSERE"
      },
      {
        "nom": "Mme Françoise, Brigitte, Jeanne, Julia LAMAILLE épouse MASSERE"
      }
    ],
    "representation_demandeurs": "SCP MORIVAL AMISSE MABIRE, Avocats associés au Barreau de DIEPPE",
    "defendeurs": [
      {
        "nom": "S.A.R.L. BRUGOT XAVIER",
        "representation": "SCP LENGLET, MALBESIN & Associés, Avocats associés au Barreau de ROUEN"
      },
      {
        "nom": "S.A. AXA France IARD",
        "representation": "SCP LENGLET, MALBESIN & Associés, Avocats associés au Barreau de ROUEN"
      },
     

## OCR

In [ ]:
def ocr_pdf(pdf_path, pages : Optional[List[int]] = None, language='eng', dpi=300) -> List[str]:
    """
    Performs OCR on a PDF and saves the text to a file.
    
    Args:
        pdf_path (str): Path to the PDF file
        output_path (str, optional): Path to save the output text. If None, uses the PDF name with .txt extension
        language (str, optional): Language for OCR. Default is 'eng'
        dpi (int, optional): DPI for rendering PDF. Higher is better quality but slower.
    """
    
    # Create temp directory for storing images
    with tempfile.TemporaryDirectory() as temp_dir:
        
        # Convert PDF to images
        try:
            images = convert_from_path(pdf_path, dpi=dpi)
            print(f"PDF converted to {len(images)} images.")
        except Exception as e:
            print(f"Error converting PDF: {e}")
            return None
        
        if pages is None:
            pages = list(range(len(images)))
        
        # Process each page
        text_per_page = []
        for i, image in tqdm(list(enumerate(images)), desc="OCR pages"):
            page_number = i + 1
            if not page_number in pages:
                continue
            
            # Perform OCR
            text = pytesseract.image_to_string(image, lang=language)
            text_per_page.append(text)
        
    return text_per_page

In [10]:
pdf_path = PATH_DOCS / "0- ARBORESCENCE DOSSIERS JUD/EXPERT/01 - Notes aux parties/Note n┬░1 - Visio Adm/TJ DIEPPE - MASSERE - Ordonnance du 05 06 2024.pdf"
assert pdf_path.exists()

In [11]:
text_per_page = ocr_pdf(pdf_path=pdf_path)
text = text_per_page[0]

PDF converted to 10 images.


OCR pages:   0%|          | 0/10 [00:00<?, ?it/s]

OCR processing page 1/10...


OCR pages:  10%|█         | 1/10 [00:02<00:22,  2.49s/it]

OCR processing page 2/10...


...

OCR pages:  70%|███████   | 7/10 [00:14<00:06,  2.14s/it]

OCR processing page 8/10...


OCR pages:  80%|████████  | 8/10 [00:16<00:04,  2.33s/it]

OCR processing page 9/10...


OCR pages: 100%|██████████| 10/10 [00:19<00:00,  1.92s/it]


In [176]:
text = "\n---------------------------------------\n".join(text_per_page[:1])
print(text)

TRIBUNAL JUDICIAIRE

DE DIEPPE

Service chargé du contr6éle des expertises judiciaires
Tél. : 02.76.72.97.34

Référence 4 rappeler dans tous courriers :

Mesure d’instruction : 24/173

Monsieur Franck HIBON
N° de R.G. : 24/00052 6 Impasse des Hautes Terres
N° Portalis DB2U-W-B7I-DDMR Les Vertus
franck.hibon @expert-de-justice.org
Ordonnance de Référé du 05 Juin 2024 76550 SAINT AUBIN SUR SCIE
DEMANDEURS EXPERTISE JUDICIAIRE, AVEC CONSIGNATION
M. Claude MASSERE
Mme Francoise, Brigitte, Jeanne, Julia Date limite de dépét du rapport d’expertise judiciaire :

LAMAILLE épouse MASSERE

28 Février 2025

représentés par la SCP MORIVAL AMISSE
MABIRE, Avocats associés au Barreau de DIEPPE

DEFENDEURS

La S.A.R.L. BRUGOT XAVIER

représentée par la SCP LENGLET, MALBESIN &
Associés, Avocats associés au Barreau de ROUEN

La S.A. AXA France IARD

représentée par la SCP LENGLET, MALBESIN &
Associés, Avocats associés au Barreau de ROUEN

M. Olivier BOUDET

représenté par la SELARL PATRICE LEMIEGRE
PHIL

## XLSX

In [13]:
path_input = PATH_TEMPLATE / "feuille_présence.xlsx"
path_output = PATH_OUTPUTS / "feuille_presence.xlsx"
assert path_input.exists() and path_output.parent.exists()

In [141]:
import openpyxl.cell.rich_text

In [ ]:
class ExcelManager:

    def __init__(self, path_excel):
        self.wb = load_workbook(path_excel, rich_text=True)
        self.ws = self.wb.worksheets[0]

    @staticmethod
    def _change(input : str, key : str, value : str, count : int) -> Tuple[str, int]:
        output = input.replace("{" + key + "}", value)
        if output != input:
            count += 1
        return output, count

    def replace_content(self, infos : dict, verbose : bool = True) -> None:
        n = 100

        changes = 0

        for row, column in itertools.product(range(1, n), range(1, n)):
            # read
            cell_value = self.ws.cell(row, column).value
            if cell_value is None:
                continue

            if isinstance(cell_value, str):
                # replace
                for key, value in infos.items():
                    cell_value, changes = ExcelManager._change(cell_value, key, value, changes)

                # write
                self.ws.cell(row, column, value=cell_value)
            elif isinstance(cell_value, openpyxl.cell.rich_text.CellRichText):
                # replace
                for i, e in enumerate(cell_value):
                    text = e if isinstance(e, str) else e.text
                    
                    for key, value in infos.items():
                        text, changes = ExcelManager._change(text, key, value, changes)

                    if isinstance(e, str):
                        cell_value[i] = text
                    else:
                        e.text = text

                    self.ws.cell(row, column, value=cell_value)
            else:
                raise ValueError(f"Excel cell type not implemmented : {type(cell_value)}")

    
        if verbose:
            print(f"Excel changes : {changes}")

    def save(self, path_excel) -> None:
        self.wb.save(path_excel)

In [171]:
infos = {
    "date_ordonnance" : "date bar",
    "numero_rg" : "rg foo",
    "lieu_expertise" : "lieu toto",
    "date_reunion" : "date baz"
}

In [172]:
excel = ExcelManager(path_input)

In [173]:
excel.replace_content(infos)
excel.save(path_output)

Excel changes : 4


## Test

In [180]:
print(text)

TRIBUNAL JUDICIAIRE

DE DIEPPE

Service chargé du contr6éle des expertises judiciaires
Tél. : 02.76.72.97.34

Référence 4 rappeler dans tous courriers :

Mesure d’instruction : 24/173

Monsieur Franck HIBON
N° de R.G. : 24/00052 6 Impasse des Hautes Terres
N° Portalis DB2U-W-B7I-DDMR Les Vertus
franck.hibon @expert-de-justice.org
Ordonnance de Référé du 05 Juin 2024 76550 SAINT AUBIN SUR SCIE
DEMANDEURS EXPERTISE JUDICIAIRE, AVEC CONSIGNATION
M. Claude MASSERE
Mme Francoise, Brigitte, Jeanne, Julia Date limite de dépét du rapport d’expertise judiciaire :

LAMAILLE épouse MASSERE

28 Février 2025

représentés par la SCP MORIVAL AMISSE
MABIRE, Avocats associés au Barreau de DIEPPE

DEFENDEURS

La S.A.R.L. BRUGOT XAVIER

représentée par la SCP LENGLET, MALBESIN &
Associés, Avocats associés au Barreau de ROUEN

La S.A. AXA France IARD

représentée par la SCP LENGLET, MALBESIN &
Associés, Avocats associés au Barreau de ROUEN

M. Olivier BOUDET

représenté par la SELARL PATRICE LEMIEGRE
PHIL

In [178]:
system = """
    Extrait toutes les informations relatives aux différents partis.
    Renvoie tes réponses sous format json :
    {
        "date_ordonnance" : "...",
        "numero_rg" : "...",
        "lieu_expertise" : "...",
        "date_reunion" : "..."
        demandeurs : [
            "nom" : "...",
            "telephone" : "...",
            "email" : "...",
            "conseil_nom" : "...",
            "conseil_telephone" : "...",
            "conseil_email" : "...",
        ],
        defendeuresses : [
            "nom" : "...",
            "telephone" : "...",
            "email" : "...",
            "conseil_nom" : "...",
            "conseil_telephone" : "...",
            "conseil_email" : "...",
            "conseil_techinque_nom" : "...",
            "conseil_techinque_telephone" : "...",
            "conseil_techinque_email" : "...",
        ]
    }
"""

messages = [
    {"role": "user", "content": text}
]

response = client.create_message(
    model="claude-3-7-sonnet-20250219",
    messages=messages,
    system=system,
    max_tokens=1000,
    temperature=0.7
)

# Print the response
print("Claude's response:")
print(response["content"][0]["text"])

Claude's response:
```json
{
    "date_ordonnance": "05 Juin 2024",
    "numero_rg": "24/00052",
    "lieu_expertise": null,
    "date_reunion": null,
    "demandeurs": [
        {
            "nom": "M. Claude MASSERE",
            "telephone": null,
            "email": null,
            "conseil_nom": "SCP MORIVAL AMISSE MABIRE, Avocats associés au Barreau de DIEPPE",
            "conseil_telephone": null,
            "conseil_email": null
        },
        {
            "nom": "Mme Francoise, Brigitte, Jeanne, Julia LAMAILLE épouse MASSERE",
            "telephone": null,
            "email": null,
            "conseil_nom": "SCP MORIVAL AMISSE MABIRE, Avocats associés au Barreau de DIEPPE",
            "conseil_telephone": null,
            "conseil_email": null
        }
    ],
    "defendeuresse": [
        {
            "nom": "S.A.R.L. BRUGOT XAVIER",
            "telephone": null,
            "email": null,
            "conseil_nom": "SCP LENGLET, MALBESIN & Associés, Avocat

In [175]:
text_infos = response["content"][0]["text"]
res = re.search(pattern="```json(.*)```", string=text_infos, flags=re.DOTALL)
infos = json.loads(res.group(1))
infos

{'date_ordonnance': '05 Juin 2024',
 'numero_rg': '24/00052',
 'lieu_expertise': '15 Rue des Bords de Mer BELLEVILLE SUR MER - 76370 PETIT-CAUX',
 'date_reunion': None,
 'demandeurs': [{'nom': 'Claude MASSERE',
   'telephone': None,
   'email': None,
   'conseil_nom': 'Maitre Corinne MORIVAL de la SCP MORIVAL AMISSE MABIRE',
   'conseil_telephone': None,
   'conseil_email': None},
  {'nom': 'Francoise Brigitte, Jeanne, Julia LAMAILLE épouse MASSERE',
   'telephone': None,
   'email': None,
   'conseil_nom': 'Maitre Corinne MORIVAL de la SCP MORIVAL AMISSE MABIRE',
   'conseil_telephone': None,
   'conseil_email': None}]}

In [22]:
excel = ExcelManager(path_input)
excel.replace_content(infos)
excel.save(path_output)

Excel changes : 4
